In [19]:
import pandas as pd
import sqlite3
import json

# Load SQL data
# Assuming the SQL file contains data creation and insertion queries
conn = sqlite3.connect('structured_data.db')  # You can use any SQL database (MySQL, PostgreSQL)

# Drop the table if it exists
conn.execute('DROP TABLE IF EXISTS structured_data')

# Execute the SQL queries to create and insert the data
with open('structured_data.sql', 'r') as file:
    sql_queries = file.read()

conn.executescript(sql_queries)

            # Now load the data into a pandas DataFrame
df_sql = pd.read_sql('SELECT * FROM structured_data', conn)


In [20]:
# Load JSON data
with open('/content/unstructured_data.json', 'r') as file:
    df_json = pd.json_normalize(json.load(file))

In [22]:
df_sql['score'] = df_sql['score'].fillna(df_sql['score'].mean())


In [23]:
# Use a regex pattern to extract the date from the log column
df_json['log'] = df_json['log'].str.extract(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})')[0]

# Convert the extracted date strings to datetime format
df_json['log'] = pd.to_datetime(df_json['log'], errors='coerce')


In [24]:
# Fill missing dates with a placeholder (optional)
df_json['log'].fillna(pd.to_datetime('2025-01-01'), inplace=True)


<ipython-input-24-99f9622ee338>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_json['log'].fillna(pd.to_datetime('2025-01-01'), inplace=True)


In [25]:
# Merge the two DataFrames on a common field (user_id and log_id)
merged_df = pd.merge(df_sql, df_json, how='inner', left_on='user_id', right_on='log_id')

# Save the merged DataFrame to a CSV file
merged_df.to_csv('merged_data.csv', index=False)

# Optionally, download the CSV file in Google Colab
from google.colab import files
files.download('merged_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>